Hand written data

In [1]:
import pandas as pd 

# Data to be used for the DataFrame
data = {
    'ipv4': ['56.73.174.168', '56.73.174.168', '56.73.174.168', '33.73.174.111', '33.73.174.111', '33.73.174.111'],
    #'ipv4': ['1', '2', '3', '4', '5', '6'],
    'scan_id': [0, 1, 2, 3, 4, 5],
    'port': [[161, 22, 25, 111, 110],[161, 33, 22, 25, 110],[161, 22, 25, 110], [22, 110, 33, 25],[22, 110, 45, 25],[22, 110, 25]],
    'service': [['SNMP', 'SSH', 'SMTP', 'POP3', 'HTTP'],['POP3', 'SNMP', 'SSH', 'HTTP', 'SMTP'],['SNMP', 'SMTP', 'HTTP' 'POP3'], ['SSH', 'HSS', 'POP3', 'SMTP'],['SSH', 'POP3', 'X', 'SMTP'],['SSH', 'POP3', 'SMTP']]
}

#df = pd.DataFrame(data)
#print(df)


Scenario 1: Increase of port usage

In [24]:
from importlib import reload
import generate_data_functions
reload(generate_data_functions)
from generate_data_functions import *

from importlib import reload
import grouped_iForest_functions
reload(grouped_iForest_functions)
from grouped_iForest_functions import *

from sklearn.ensemble import IsolationForest
from statistics import mean


# Example directory path
directory_path = '/Users/christiedjidjev/Censys Pages'
service_port_dict = generate_service_port_dict(directory_path)

print('service_port_dict:', service_port_dict)

# Calculate distributions
norm_service_distribution, norm_port_distributions = calculate_distributions(service_port_dict)

# Reasonable test values as class variables
class DataParameters:
    num_ips = 10
    num_scans_per_ip = 20
    norm_list_length_distribution = [0.1, 0.2, 0.3, 0.2, 0.1, 0.05, 0.05]
    anom_list_length_distribution = [0, 0, 0, 0, 0, 0, 0, 0, 0.05, 0.05, 0.1, 0.1, 0.2, 0.2, 0.3]
    
    anom_service_distribution = norm_service_distribution
    anom_port_distribution = norm_port_distributions
    anomaly_rate = 0.2

# List of seeds
num_repetitions = 10
seeds = range(num_repetitions)

# List to store all experiments
all_exp = []

# Loop over seeds
for seed in seeds:
    # Set the random seed for reproducibility
    np.random.seed(seed)
    
    # Generate the data
    df = generate_scan_data_1(
        DataParameters.num_ips,
        DataParameters.num_scans_per_ip,
        DataParameters.norm_list_length_distribution,
        norm_service_distribution,
        norm_port_distributions,
        DataParameters.anom_list_length_distribution,
        DataParameters.anom_service_distribution,
        DataParameters.anom_port_distribution,
        DataParameters.anomaly_rate,
        seed
    )

    anomalous_ips = df[df['anomaly'] == 1]['ipv4'].unique()
    print('anomalous_ips:', anomalous_ips)

    # Run Data Version (original - flattened) 1
    print('1 FLAT - GROUPED')
    features_flat = preprocess_customiForest(df)
    custForest_flat = CustomIsolationForest(ipv4_index=0, n_estimators=2, random_state=0, df=df)
    custForest_flat.fit(features_flat)
    custForest_flat_labels = custForest_flat.predict(features_flat, min_score=False)
    custForest_flat_scores = custForest_flat.decision_function(features_flat)
    
    regForest_flat = IsolationForest(random_state=0)
    regForest_flat.fit(features_flat)
    regForest_flat_labels = regForest_flat.predict(features_flat)
    regForest_flat_scores = regForest_flat.decision_function(features_flat)    
    features_flat['anomaly_score'] = regForest_flat_scores

    # Group by 'ipv4' and sum anomaly scores
    features_flat['ipv4'] = features_flat[['octet1', 'octet2', 'octet3', 'octet4']].astype(str).agg('.'.join, axis=1)
    anomalies_sum = features_flat.groupby('ipv4')['anomaly_score'].sum().reset_index()
    anomalies_count = features_flat['ipv4'].value_counts().reset_index(name='count')
    merged = pd.merge(anomalies_sum, anomalies_count, on='ipv4')
    anomalies_avg = merged['anomaly_score'] / merged['count']

    # Sort anomalies by their average score
    #sorted_anomalies_avg = anomalies_avg.sort_values()

    # Create the dictionary to store the results
    reg_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for ipv4, anomaly_score in anomalies_avg.items():
        reg_anomaly_dict[ipv4] = anomaly_score

    # Run Data Version (IP-port pairs) 2
    print('2 PAIRS')

    df_pairs = pairs(df)
    #print("df_pairs", df_pairs)
    # Display the feature matrix
    all_columns = df_pairs.columns.tolist()

    #Specify the column to exclude
    column_to_exclude = ['ipv4', 'anomaly']

    # Create a new list excluding the specified column
    features = [col for col in all_columns if col not in column_to_exclude]
    #print('features:', features)
    regForest_pairs = IsolationForest(random_state=0)
    regForest_pairs.fit(df_pairs[features])
    regForest_pairs_labels = regForest_pairs.predict(df_pairs[features])
    regForest_pairs_scores = regForest_pairs.decision_function(df_pairs[features])  
    df_pairs['anomaly_score'] = regForest_pairs_scores

    # Group by 'ipv4' and sum anomaly scores
    anomalies_sum = df_pairs.groupby('ipv4')['anomaly_score'].sum()
    anomalies_count = df_pairs['ipv4'].value_counts()

    anomalies_avg = anomalies_sum / anomalies_count

    # Sort anomalies by their average score
    sorted_anomalies_avg = anomalies_avg.sort_values()

    # Create the dictionary to store the results
    pairs_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for ipv4, anomaly_score in sorted_anomalies_avg.items():
        pairs_anomaly_dict[ipv4] = anomaly_score
        
    # Run Data Version (summarization) 3
    print('3 SUM')
    
    df_sum = summarize_features(df)
    df_sum= df_sum.rename(str,axis="columns") 

    #print(df_sum)
    all_columns = df_sum.columns
    column_to_exclude = ['ipv4', 'anomaly']
    features = [col for col in df_sum.columns if col not in column_to_exclude]
    
    #df_sum[features].columns = df_sum[features].columns.astype(str)
    #print(df_sum.columns, df_sum.dtypes)

    regForest_sum = IsolationForest(random_state=0)
    regForest_sum.fit(df_sum[features])

    regForest_sum_labels = regForest_sum.predict(df_sum[features])
    regForest_sum_scores = regForest_sum.decision_function(df_sum[features])
    df_sum['anomaly_score'] = regForest_sum_scores

    sum_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for index, row in df_sum.iterrows():
        ipv4 = row['ipv4']
        anomaly_score = row['anomaly_score']
        sum_anomaly_dict[ipv4] = anomaly_score


    # Anomalous IPs from the Custom Isolation Forest (Flattening)
    custForest_anomalous_ips = [(ip, score) for ip, score in custForest_flat_scores.items() if score < 0]
    # Anomalous IPs from the Standard Isolation Forest (Flattening)
    #print('reg_anomaly_dict:', reg_anomaly_dict)
    regForest_anomalous_ips = [(ip, score) for ip, score in reg_anomaly_dict.items() if score < 0]
    # Standard Isolation Forest (IP-Port Pairs): (IP, score) pairs for anomalous IPs
    pairs_anomalous_ips = [(ip, score) for ip, score in pairs_anomaly_dict.items() if score < 0]
    # Standard Isolation Forest (Summarization): (IP, score) pairs for anomalous IPs
    sum_anomalous_ips = [(ip, score) for ip, score in sum_anomaly_dict.items() if score < 0]

    def calculate_f2_score(true_anomalous_ips, predicted_anomalous):
        """
        Calculates the F2-Score based on true anomalous IPs and predicted anomalous IPs.
        
        Parameters:
        - true_anomalous_ips (list): List of true anomalous IP addresses.
        - predicted_anomalous (list of tuples): List of (IP, score) for IPs flagged as anomalous.
        
        Returns:
        - float: The F2-Score for the model.
        """
        
        # Convert inputs to sets of IPs for comparison
        true_anomalous_set = set(true_anomalous_ips)
        predicted_anomalous_set = set(ip for ip, score in predicted_anomalous)
        
        # Calculate true positives, false positives, and false negatives
        tp = len(true_anomalous_set & predicted_anomalous_set)
        fp = len(predicted_anomalous_set - true_anomalous_set)
        fn = len(true_anomalous_set - predicted_anomalous_set)
        
        # Calculate precision and recall
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        # Calculate F2-Score with beta = 2
        f2_score = (1 + 2**2) * (precision * recall) / (2**2 * precision + recall) if (precision + recall) > 0 else 0
        
        return f2_score

    # Main code
    # Assuming anomalous_ips contains the true anomalous IPs
    # Calling the function for each iForest version and printing the F2-Score

    # F2-Score for Custom iForest
    f2_score_custom = calculate_f2_score(anomalous_ips, custForest_anomalous_ips)

    # F2-Score for Custom iForest
    f2_score_reg = calculate_f2_score(anomalous_ips, regForest_anomalous_ips)

    # F2-Score for iForest Pairs
    f2_score_pairs = calculate_f2_score(anomalous_ips, pairs_anomalous_ips)

    # F2-Score for iForest Summarization
    f2_score_sum = calculate_f2_score(anomalous_ips, sum_anomalous_ips)

    # Step 3: Store all information in the experiment dictionary
    exp = {
        'seed': seed,
        'num_ips': DataParameters.num_ips,
        'num_scans_per_ip': DataParameters.num_scans_per_ip,
        "Custom iForest Flattening scores": custForest_flat_scores,
        "Standard iForest Flattening scores": regForest_flat_scores,
        "iForest Pairs scores": pairs_anomaly_dict,
        "iForest Summarization scores": sum_anomaly_dict,
        "Actual Anomalous IPs": anomalous_ips,
        "Custom iForest Anomalous IPs": custForest_anomalous_ips,
        "iForest Pairs Anomalous IPs": pairs_anomalous_ips,
        "iForest Summarization Anomalous IPs": sum_anomalous_ips,
        "Custom iForest F2-Score": f2_score_custom,
        "Standard iForest F2-Score": f2_score_reg,
        "iForest Pairs F2-Score": f2_score_pairs,
        "iForest Summarization F2-Score": f2_score_sum

    }
    
    # Add the experiment dictionary to the list of all experiments
    all_exp.append(exp)

# Print all experiments for verification
'''for experiment in all_exp:
    for key, value in experiment.items():
        print(f"{key}: {value}")
    print("-" * 20)  # Separator between experimentsK)'''

# Calculate average F2 scores


f2_scores_custom = [experiment["Custom iForest F2-Score"] for experiment in all_exp]
f2_scores_reg = [experiment["Standard iForest F2-Score"] for experiment in all_exp]
f2_scores_pairs = [experiment["iForest Pairs F2-Score"] for experiment in all_exp]
f2_scores_sum = [experiment["iForest Summarization F2-Score"] for experiment in all_exp]

print(f2_scores_custom, f2_scores_reg, f2_scores_pairs, f2_scores_sum)
avg_f2_score_custom = mean(f2_scores_custom)
avg_f2_score_reg = mean(f2_scores_reg)
avg_f2_score_pairs = mean(f2_scores_pairs)
avg_f2_score_sum = mean(f2_scores_sum)


# Print the average F2 scores
print(f"Average Custom iForest F2-Score: {avg_f2_score_custom}")
print(f"Average Standard iForest F2-Score: {avg_f2_score_reg}")
print(f"Average iForest Pairs F2-Score: {avg_f2_score_pairs}")
print(f"Average iForest Summarization F2-Score: {avg_f2_score_sum}")


service_port_dict: defaultdict(<function generate_service_port_dict.<locals>.<lambda> at 0x125ca4280>, {'FTP': defaultdict(<class 'int'>, {'21': 140, '2121': 1, '990': 3}), 'SMTP': defaultdict(<class 'int'>, {'25': 140, '26': 38, '465': 121, '587': 119, '2525': 9, '2626': 1, '225': 2, '10443': 1, '10821': 1, '11523': 1, '12215': 1, '15711': 1, '17080': 1, '7025': 1, '902': 1}), 'DNS': defaultdict(<class 'int'>, {'53': 99}), 'HTTP': defaultdict(<class 'int'>, {'80': 175, '443': 166, '2082': 72, '2083': 75, '2086': 67, '2087': 68, '2095': 71, '2096': 68, '8083': 1, '2077': 63, '2078': 65, '2091': 13, '7080': 6, '7081': 6, '8443': 23, '8880': 18, '2079': 23, '2080': 24, '1500': 6, '81': 5, '9001': 3, '10000': 2, '20000': 1, '8429': 1, '8888': 7, '5357': 2, '7547': 6, '8043': 1, '8080': 15, '8081': 4, '8088': 3, '8090': 5, '8843': 1, '1050': 1, '2200': 1, '3000': 2, '4060': 1, '5500': 1, '6070': 1, '7400': 1, '8000': 8, '8060': 1, '9100': 4, '3128': 2, '2': 3, '2052': 1, '2053': 1, '82': 2

Scenario 2: Unusual ports used

In [3]:
from importlib import reload
import generate_data_functions
reload(generate_data_functions)
from generate_data_functions import *

from importlib import reload
import grouped_iForest_functions
reload(grouped_iForest_functions)
from grouped_iForest_functions import *

from sklearn.ensemble import IsolationForest

# Example directory path
directory_path = '/Users/christiedjidjev/Censys Pages'
typical_services_ports = generate_service_port_dict(directory_path)  # Creates dictionary like {'service1': 'port1', ...}

class DataParameters:
    num_ips = 10
    num_scans_per_ip = 20
    norm_list_length_distribution = [0.1, 0.2, 0.3, 0.2, 0.1, 0.05, 0.05]
    anom_list_length_distribution = [0, 0, 0, 0, 0, 0, 0, 0, 0.05, 0.05, 0.1, 0.1, 0.2, 0.2, 0.3]
    
    anom_service_distribution = norm_service_distribution
    anom_port_distribution = norm_port_distributions
    anomaly_rate = 0.2

# List of seeds
num_repetitions = 10
seeds = range(num_repetitions)

# List to store all experiments
all_exp = []

# Loop over seeds
for seed in seeds:
    # Set the random seed for reproducibility
    np.random.seed(seed)
    
    # Generate the data
    df = generate_scan_data(
        DataParameters.num_ips,
        DataParameters.num_scans_per_ip,
        DataParameters.norm_list_length_distribution,
        norm_service_distribution,
        norm_port_distributions,
        DataParameters.anom_list_length_distribution,
        DataParameters.anom_service_distribution,
        DataParameters.anom_port_distribution,
        DataParameters.anomaly_rate,
        seed
    )

    anomalous_ips = df[df['anomaly'] == 1]['ipv4'].unique()
    print('anomalous_ips:', anomalous_ips)

    # Run Data Version (original - flattened) 1
    print('1 FLAT - GROUPED')
    features_flat = preprocess_customiForest(df)
    custForest_flat = CustomIsolationForest(ipv4_index=0, n_estimators=2, random_state=0, df=df)
    custForest_flat.fit(features_flat)
    custForest_flat_labels = custForest_flat.predict(features_flat, min_score=False)
    custForest_flat_scores = custForest_flat.decision_function(features_flat)

    regForest_flat = IsolationForest(random_state=0)
    regForest_flat.fit(features_flat)
    regForest_flat_labels = regForest_flat.predict(features_flat)
    regForest_flat_scores = regForest_flat.decision_function(features_flat)    

    # Run Data Version (IP-port pairs) 2
    print('2 PAIRS')

    df_pairs = pairs(df)
    #print("df_pairs", df_pairs)
    # Display the feature matrix
    all_columns = df_pairs.columns.tolist()

    #Specify the column to exclude
    column_to_exclude = ['ipv4', 'anomaly']

    # Create a new list excluding the specified column
    features = [col for col in all_columns if col not in column_to_exclude]
    #print('features:', features)
    regForest_pairs = IsolationForest(random_state=0)
    regForest_pairs.fit(df_pairs[features])
    regForest_pairs_labels = regForest_pairs.predict(df_pairs[features])
    regForest_pairs_scores = regForest_pairs.decision_function(df_pairs[features])  
    df_pairs['anomaly_score'] = regForest_pairs_scores

    # Group by 'ipv4' and sum anomaly scores
    anomalies_sum = df_pairs.groupby('ipv4')['anomaly_score'].sum()
    anomalies_count = df_pairs['ipv4'].value_counts()
    anomalies_avg = anomalies_sum / anomalies_count

    # Sort anomalies by their average score
    sorted_anomalies_avg = anomalies_avg.sort_values()

    # Create the dictionary to store the results
    pairs_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for ipv4, anomaly_score in sorted_anomalies_avg.items():
        pairs_anomaly_dict[ipv4] = anomaly_score
        
    # Run Data Version (summarization) 3
    print('3 SUM')
    
    df_sum = summarize_features(df)
    df_sum= df_sum.rename(str,axis="columns")
    #print(df_sum)

    all_columns = df_sum.columns.tolist()
    column_to_exclude = ['ipv4', 'anomaly']
    features = [col for col in all_columns if col not in column_to_exclude]

    regForest_sum = IsolationForest(random_state=0)
    regForest_sum.fit(df_sum[features])
    regForest_sum_labels = regForest_sum.predict(df_sum[features])
    regForest_sum_scores = regForest_sum.decision_function(df_sum[features])
    df_sum['anomaly_score'] = regForest_sum_scores

    sum_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for index, row in df_sum.iterrows():
        ipv4 = row['ipv4']
        anomaly_score = row['anomaly_score']
        sum_anomaly_dict[ipv4] = anomaly_score


    # Anomalous IPs from the Custom Isolation Forest (Flattening)
    custForest_anomalous_ips = [(ip, score) for ip, score in custForest_flat_scores.items() if score < 0]

    # Standard Isolation Forest (IP-Port Pairs): (IP, score) pairs for anomalous IPs
    pairs_anomalous_ips = [(ip, score) for ip, score in pairs_anomaly_dict.items() if score < 0]

    # Standard Isolation Forest (Summarization): (IP, score) pairs for anomalous IPs
    sum_anomalous_ips = [(ip, score) for ip, score in sum_anomaly_dict.items() if score < 0]

    def calculate_f2_score(true_anomalous_ips, predicted_anomalous):
        """
        Calculates the F2-Score based on true anomalous IPs and predicted anomalous IPs.
        
        Parameters:
        - true_anomalous_ips (list): List of true anomalous IP addresses.
        - predicted_anomalous (list of tuples): List of (IP, score) for IPs flagged as anomalous.
        
        Returns:
        - float: The F2-Score for the model.
        """
        
        # Convert inputs to sets of IPs for comparison
        true_anomalous_set = set(true_anomalous_ips)
        predicted_anomalous_set = set(ip for ip, score in predicted_anomalous)
        
        # Calculate true positives, false positives, and false negatives
        tp = len(true_anomalous_set & predicted_anomalous_set)
        fp = len(predicted_anomalous_set - true_anomalous_set)
        fn = len(true_anomalous_set - predicted_anomalous_set)
        
        # Calculate precision and recall
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        # Calculate F2-Score with beta = 2
        f2_score = (1 + 2**2) * (precision * recall) / (2**2 * precision + recall) if (precision + recall) > 0 else 0
        
        return f2_score

    # Main code
    # Assuming anomalous_ips contains the true anomalous IPs
    # Calling the function for each iForest version and printing the F2-Score

    # F2-Score for Custom iForest
    f2_score_custom = calculate_f2_score(anomalous_ips, custForest_anomalous_ips)

    # F2-Score for iForest Pairs
    f2_score_pairs = calculate_f2_score(anomalous_ips, pairs_anomalous_ips)

    # F2-Score for iForest Summarization
    f2_score_sum = calculate_f2_score(anomalous_ips, sum_anomalous_ips)

    # Step 3: Store all information in the experiment dictionary
    exp = {
        'seed': seed,
        'num_ips': DataParameters.num_ips,
        'num_scans_per_ip': DataParameters.num_scans_per_ip,
        "Custom iForest Flattening scores": custForest_flat_scores,
        "iForest Pairs scores": pairs_anomaly_dict,
        "iForest Summarization scores": sum_anomaly_dict,
        "Actual Anomalous IPs": anomalous_ips,
        "Custom iForest Anomalous IPs": custForest_anomalous_ips,
        "iForest Pairs Anomalous IPs": pairs_anomalous_ips,
        "iForest Summarization Anomalous IPs": sum_anomalous_ips,
        "Custom iForest F2-Score": f2_score_custom,
        "iForest Pairs F2-Score": f2_score_pairs,
        "iForest Summarization F2-Score": f2_score_sum
    }
    
    # Add the experiment dictionary to the list of all experiments
    all_exp.append(exp)

# Print all experiments for verification
'''
for experiment in all_exp:
    for key, value in experiment.items():
        print(f"{key}: {value}")
    print("-" * 20)  # Separator between experiments
    '''

f2_scores_custom = [experiment["Custom iForest F2-Score"] for experiment in all_exp]
f2_scores_pairs = [experiment["iForest Pairs F2-Score"] for experiment in all_exp]
f2_scores_sum = [experiment["iForest Summarization F2-Score"] for experiment in all_exp]

print(f2_scores_sum, f2_scores_pairs, f2_scores_custom)
avg_f2_score_custom = mean(f2_scores_custom)
avg_f2_score_pairs = mean(f2_scores_pairs)
avg_f2_score_sum = mean(f2_scores_sum)


# Print the average F2 scores
print(f"Average Custom iForest F2-Score: {avg_f2_score_custom}")
print(f"Average iForest Pairs F2-Score: {avg_f2_score_pairs}")
print(f"Average iForest Summarization F2-Score: {avg_f2_score_sum}")


anomalous_ips: ['158.50.37.169' '241.51.181.222']
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: ['15.11.13.4']
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: ['166.88.69.184' '93.228.212.186' '181.185.228.82']
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: ['119.98.240.243']
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: ['245.79.46.34' '133.139.99.84' '172.198.127.91']
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: ['148.160.101.104']
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: ['227.48.100.149']
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: ['165.77.202.24' '37.48.187.29']
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: []
1 FLAT - GROUPED
2 PAIRS
3 SUM
anomalous_ips: ['237.191.136.70']
1 FLAT - GROUPED
2 PAIRS
3 SUM
[1.0, 1.0, 0.7142857142857142, 0, 0.3846153846153846, 1.0, 1.0, 1.0, 0, 1.0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0.3846153846153846, 0.5555555555555556, 0.5882352941176471, 0.5555555555555556, 0.5882352941176471, 0.5555555555555556, 0.5555555555555556, 0.7692307692307692, 0, 0.5

In [4]:
typical_services_ports

defaultdict(<function generate_data_functions.generate_service_port_dict.<locals>.<lambda>()>,
            {'FTP': defaultdict(int, {'21': 140, '2121': 1, '990': 3}),
             'SMTP': defaultdict(int,
                         {'25': 140,
                          '26': 38,
                          '465': 121,
                          '587': 119,
                          '2525': 9,
                          '2626': 1,
                          '225': 2,
                          '10443': 1,
                          '10821': 1,
                          '11523': 1,
                          '12215': 1,
                          '15711': 1,
                          '17080': 1,
                          '7025': 1,
                          '902': 1}),
             'DNS': defaultdict(int, {'53': 99}),
             'HTTP': defaultdict(int,
                         {'80': 175,
                          '443': 166,
                          '2082': 72,
                          '2083'

In [5]:
# Save DataFrame to CSV
df.to_csv('usage.csv', index=False)

Scenario 3: Stray from typical

In [7]:
from importlib import reload
import generate_data_functions
reload(generate_data_functions)
from generate_data_functions import *

from importlib import reload
import grouped_iForest_functions
reload(grouped_iForest_functions)
from grouped_iForest_functions import *

from sklearn.ensemble import IsolationForest
typical_services_ports = generate_service_port_dict(directory_path)  # Creates dictionary like {'service1': 'port1', ...}

# Reasonable test values as class variables
instances_per_service = 5
typical_service_port_map = {
    'FTP': 21,
    'SMTP': 25,
    'DNS': 53,
    'HTTP': 80
    }
    #'POP3': 110,
    #'IMAP': 143,
    #'MYSQL': 3306,
    #'SSH': 22
#}

# List of seeds
num_repetitions = 1
seeds = range(num_repetitions)

# List to store all experiments
all_exp = []

# Loop over seeds
for seed in seeds:
    # Set the random seed for reproducibility
    np.random.seed(None)
    
    num_ips=5
    num_scans_per_ip=10
    anomaly_rate=0.2
    # Generate the data
    df = generate_scan_data_with_variance(num_ips, num_scans_per_ip, typical_service_port_map, anomaly_rate, seed)
    print(df.head(100))
    '''anomalous_scans = [
    ['198.51.100.1', 0, 0, [80, 22, 21], ['HTTP', 'SSH', 'FTP'], 0],
    ['198.51.100.1', 0, 1, [143, 3306], ['IMAP', 'MYSQL'], 0],
    ['198.51.100.1', 0, 2, [80, 143], ['HTTP', 'IMAP'], 0],
    ['198.51.100.1', 0, 3, [22, 110], ['SSH', 'POP3'], 0],
    ['198.51.100.1', 0, 4, [53, 3306], ['DNS', 'MYSQL'], 0],
    ['198.51.100.1', 0, 5, [80, 3306], ['HTTP', 'MYSQL'], 1], # Anomaly: MYSQL on port 80
    ['198.51.100.1', 0, 6, [21, 143], ['FTP', 'IMAP'], 1],  # Anomaly: IMAP on port 21
    ['198.51.100.1', 0, 7, [22, 53], ['SSH', 'DNS'], 0],
    ['198.51.100.1', 0, 8, [110, 3306], ['POP3', 'MYSQL'], 0],
    ['198.51.100.1', 0, 9, [21, 143], ['FTP', 'IMAP'], 0]
]
    
    anomalous_df = pd.DataFrame(anomalous_scans, columns=['ipv4', 'ip_id', 'scan_id', 'port', 'service', 'anomaly'])'''

    #df = pd.concat([df, anomalous_df], ignore_index=True)
    anomalous_ips = df[df['anomaly'] == 1]['ipv4'].unique()
    print('anomalous_ips:', anomalous_ips)

    # Run Data Version (original - flattened) 1
    print('1 FLAT - GROUPED')
    features_flat = preprocess_customiForest(df)
    custForest_flat = CustomIsolationForest(ipv4_index=0, n_estimators=2, random_state=0, df=df)
    custForest_flat.fit(features_flat)
    custForest_flat_labels = custForest_flat.predict(features_flat, min_score=True)
    custForest_flat_scores = custForest_flat.decision_function(features_flat)

    regForest_flat = IsolationForest(random_state=0)
    regForest_flat.fit(features_flat)
    regForest_flat_labels = regForest_flat.predict(features_flat)
    regForest_flat_scores = regForest_flat.decision_function(features_flat)    

    # Run Data Version (IP-port pairs) 2
    print('2 PAIRS')

    df_pairs = pairs(df)
    #print("df_pairs", df_pairs)
    # Display the feature matrix
    all_columns = df_pairs.columns.tolist()

    #Specify the column to exclude
    column_to_exclude = ['ipv4', 'anomaly']

    # Create a new list excluding the specified column
    features = [col for col in all_columns if col not in column_to_exclude]
    #print('features:', features)
    regForest_pairs = IsolationForest(random_state=0)
    regForest_pairs.fit(df_pairs[features])
    regForest_pairs_labels = regForest_pairs.predict(df_pairs[features])
    regForest_pairs_scores = regForest_pairs.decision_function(df_pairs[features])  
    df_pairs['anomaly_score'] = regForest_pairs_scores

    # Group by 'ipv4' and sum anomaly scores
    anomalies_sum = df_pairs.groupby('ipv4')['anomaly_score'].sum()
    anomalies_count = df_pairs['ipv4'].value_counts()

    anomalies_avg = anomalies_sum / anomalies_count

    # Sort anomalies by their average score
    sorted_anomalies_avg = anomalies_avg.sort_values()

    # Create the dictionary to store the results
    pairs_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for ipv4, anomaly_score in sorted_anomalies_avg.items():
        pairs_anomaly_dict[ipv4] = anomaly_score
        
    # Run Data Version (summarization) 3
    print('3 SUM')
    
    df_sum = summarize_features(df)
    df_sum= df_sum.rename(str,axis="columns")

    print(df_sum)

    all_columns = df_sum.columns.tolist()
    column_to_exclude = ['ipv4', 'anomaly']
    features = [col for col in all_columns if col not in column_to_exclude]

    regForest_sum = IsolationForest(random_state=0)
    regForest_sum.fit(df_sum[features])
    regForest_sum_labels = regForest_sum.predict(df_sum[features])
    regForest_sum_scores = regForest_sum.decision_function(df_sum[features])
    df_sum['anomaly_score'] = regForest_sum_scores

    sum_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for index, row in df_sum.iterrows():
        ipv4 = row['ipv4']
        anomaly_score = row['anomaly_score']
        sum_anomaly_dict[ipv4] = anomaly_score


    # Anomalous IPs from the Custom Isolation Forest (Flattening)
    custForest_anomalous_ips = [(ip, score) for ip, score in custForest_flat_scores.items() if score < 0]

    # Standard Isolation Forest (IP-Port Pairs): (IP, score) pairs for anomalous IPs
    pairs_anomalous_ips = [(ip, score) for ip, score in pairs_anomaly_dict.items() if score < 0]

    # Standard Isolation Forest (Summarization): (IP, score) pairs for anomalous IPs
    sum_anomalous_ips = [(ip, score) for ip, score in sum_anomaly_dict.items() if score < 0]

    def calculate_f2_score(true_anomalous_ips, predicted_anomalous):
        """
        Calculates the F2-Score based on true anomalous IPs and predicted anomalous IPs.
        
        Parameters:
        - true_anomalous_ips (list): List of true anomalous IP addresses.
        - predicted_anomalous (list of tuples): List of (IP, score) for IPs flagged as anomalous.
        
        Returns:
        - float: The F2-Score for the model.
        """
        
        # Convert inputs to sets of IPs for comparison
        true_anomalous_set = set(true_anomalous_ips)
        predicted_anomalous_set = set(ip for ip, score in predicted_anomalous)
        
        # Calculate true positives, false positives, and false negatives
        tp = len(true_anomalous_set & predicted_anomalous_set)
        fp = len(predicted_anomalous_set - true_anomalous_set)
        fn = len(true_anomalous_set - predicted_anomalous_set)
        
        # Calculate precision and recall
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        # Calculate F2-Score with beta = 2
        f2_score = (1 + 2**2) * (precision * recall) / (2**2 * precision + recall) if (precision + recall) > 0 else 0
        
        return f2_score

    # Main code
    # Assuming anomalous_ips contains the true anomalous IPs
    # Calling the function for each iForest version and printing the F2-Score

    # F2-Score for Custom iForest
    f2_score_custom = calculate_f2_score(anomalous_ips, custForest_anomalous_ips)

    # F2-Score for iForest Pairs
    f2_score_pairs = calculate_f2_score(anomalous_ips, pairs_anomalous_ips)

    # F2-Score for iForest Summarization
    f2_score_sum = calculate_f2_score(anomalous_ips, sum_anomalous_ips)

    # Step 3: Store all information in the experiment dictionary
    exp = {
        'seed': seed,
        'num_ips': DataParameters.num_ips,
        'num_scans_per_ip': DataParameters.num_scans_per_ip,
        "Custom iForest Flattening scores": custForest_flat_scores,
        "iForest Pairs scores": pairs_anomaly_dict,
        "iForest Summarization scores": sum_anomaly_dict,
        "Actual Anomalous IPs": anomalous_ips,
        "Custom iForest Anomalous IPs": custForest_anomalous_ips,
        "iForest Pairs Anomalous IPs": pairs_anomalous_ips,
        "iForest Summarization Anomalous IPs": sum_anomalous_ips,
        "Custom iForest F2-Score": f2_score_custom,
        "iForest Pairs F2-Score": f2_score_pairs,
        "iForest Summarization F2-Score": f2_score_sum
    }
    
    # Add the experiment dictionary to the list of all experiments
    all_exp.append(exp)

# Print all experiments for verification
'''
for experiment in all_exp:
    for key, value in experiment.items():
        print(f"{key}: {value}")
    print("-" * 20)  # Separator between experiments
    '''

f2_scores_custom = [experiment["Custom iForest F2-Score"] for experiment in all_exp]
f2_scores_pairs = [experiment["iForest Pairs F2-Score"] for experiment in all_exp]
f2_scores_sum = [experiment["iForest Summarization F2-Score"] for experiment in all_exp]

print(f2_scores_sum, f2_scores_pairs, f2_scores_custom)
avg_f2_score_custom = mean(f2_scores_custom)
avg_f2_score_pairs = mean(f2_scores_pairs)
avg_f2_score_sum = mean(f2_scores_sum)


# Print the average F2 scores
print(f"Average Custom iForest F2-Score: {avg_f2_score_custom}")
print(f"Average iForest Pairs F2-Score: {avg_f2_score_pairs}")
print(f"Average iForest Summarization F2-Score: {avg_f2_score_sum}")


is_anomalous_ip True
is_anomalous_ip False
is_anomalous_ip False
is_anomalous_ip False
is_anomalous_ip False
              ipv4  ip_id  scan_id              port                 service  \
0   197.215.20.132      0        0  [25, 53, 80, 21]  [SMTP, DNS, HTTP, FTP]   
1   197.215.20.132      0        1          [80, 53]             [HTTP, DNS]   
2   197.215.20.132      0        2      [25, 21, 80]       [SMTP, FTP, HTTP]   
3   197.215.20.132      0        3  [80, 53, 25, 21]  [HTTP, DNS, SMTP, FTP]   
4   197.215.20.132      0        4  [80, 25, 53, 21]  [HTTP, SMTP, DNS, FTP]   
5   197.215.20.132      0        5  [25, 53, 21, 80]  [SMTP, DNS, FTP, HTTP]   
6   197.215.20.132      0        6  [53, 80, 21, 25]  [DNS, HTTP, FTP, SMTP]   
7   197.215.20.132      0        7  [21, 53, 80, 25]  [FTP, DNS, HTTP, SMTP]   
8   197.215.20.132      0        8  [53, 25, 80, 21]  [HTTP, SMTP, DNS, FTP]   
9   197.215.20.132      0        9  [80, 25, 21, 53]  [HTTP, SMTP, DNS, FTP]   
10   32.97.

Step 1:
Generate normal scan data

Step 2: 
Choose an arbitrary scan 

Step 3: 
In the port list, swap two random port numbers if they are different

For example:
If the list has n1 and n2 in positions 5 and 10 after the change n2 will be on position 5 and n1 will be on position 10

In [ ]:
features_flat

,ip_id,octet1,octet2,octet3,octet4,port_25,port_53,port_80,port_21,service_SMTP,service_DNS,service_HTTP,service_FTP
0,0,197,215,20,132,1,1,1,1,1,1,1,1
1,0,197,215,20,132,0,1,1,0,0,1,1,0
2,0,197,215,20,132,1,0,1,1,1,0,1,1
3,0,197,215,20,132,1,1,1,1,1,1,1,1
4,0,197,215,20,132,1,1,1,1,1,1,1,1
5,0,197,215,20,132,1,1,1,1,1,1,1,1
6,0,197,215,20,132,1,1,1,1,1,1,1,1
7,0,197,215,20,132,1,1,1,1,1,1,1,1
8,0,197,215,20,132,1,1,1,1,1,1,1,1
9,0,197,215,20,132,1,1,1,1,1,1,1,1


In [ ]:
df.head(100)

,ipv4,ip_id,scan_id,port,service,anomaly
0,197.215.20.132,0,0,"[25, 53, 80, 21]","[SMTP, DNS, HTTP, FTP]",0
1,197.215.20.132,0,1,"[80, 53]","[HTTP, DNS]",0
2,197.215.20.132,0,2,"[25, 21, 80]","[SMTP, FTP, HTTP]",0
3,197.215.20.132,0,3,"[80, 53, 25, 21]","[HTTP, DNS, SMTP, FTP]",0
4,197.215.20.132,0,4,"[80, 25, 53, 21]","[HTTP, SMTP, DNS, FTP]",0
5,197.215.20.132,0,5,"[25, 53, 21, 80]","[SMTP, DNS, FTP, HTTP]",0
6,197.215.20.132,0,6,"[53, 80, 21, 25]","[DNS, HTTP, FTP, SMTP]",0
7,197.215.20.132,0,7,"[21, 53, 80, 25]","[FTP, DNS, HTTP, SMTP]",0
8,197.215.20.132,0,8,"[53, 25, 80, 21]","[HTTP, SMTP, DNS, FTP]",1
9,197.215.20.132,0,9,"[80, 25, 21, 53]","[HTTP, SMTP, DNS, FTP]",1
